In [1]:
# TODO: remove cell
from articlecommon import *
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder
lgbm = LGBMClassifier(random_state=42, verbose=-1)
psc_ohe = pd.get_dummies(X_le["Policy_Sales_Channel"], prefix="Policy_Sales_Channel")
X_le_psc = pd.concat([X_le, psc_ohe], axis=1)



psc_ohe = pd.get_dummies(X_le["Policy_Sales_Channel"], prefix="Policy_Sales_Channel")

counts = X_le["Policy_Sales_Channel"].value_counts()
counts = counts[counts == 1]
columns_to_remove = list(map(lambda x: "Policy_Sales_Channel_" + str(x), counts.index.tolist()))
psc_ohe = psc_ohe.drop(columns=columns_to_remove)

X_le_psc = pd.concat([X_le, psc_ohe], axis=1)

value_counts = X_le_psc["Policy_Sales_Channel"].value_counts().index
columns_to_remove = list(map(lambda x: "Policy_Sales_Channel_" + str(value_counts[x]), [*range(147, 86, -1)]))
columns_to_remove += ['Vintage']

from category_encoders import TargetEncoder
X_le_psc_te = X_le_psc.drop(columns=columns_to_remove).copy()
X_le_psc_te['Policy_Sales_Channel_te'] = X_le_psc_te['Policy_Sales_Channel']

X_le_psc_te_rc = X_le_psc_te.copy()
X_le_psc_te_rc['Region_Code_te'] = X_le_psc_te_rc['Region_Code']
selected_to_remove = pd.read_csv('results/columns_to_remov4.csv', index_col=0)["0"]

X_final = X_le_psc_te_rc
X_rc_ohe = X_final.drop(columns=selected_to_remove)
rc_ohe = pd.get_dummies(X_rc_ohe["Region_Code"], prefix="Region_Code")
X_rc_ohe = pd.concat([X_rc_ohe, rc_ohe], axis=1)

fixed_columns = [x for x in X_rc_ohe if x not in rc_ohe.columns]
columns_to_add = ["Region_Code_0.0", "Region_Code_1.0", "Region_Code_38.0", "Region_Code_52.0"]

encoder = TargetEncoder(cols=['Region_Code', 'Policy_Sales_Channel'])
# X_reduce_te = X_rc_ohe[fixed_columns + columns_to_add]
X_reduce_te = X_rc_ohe.loc[:, fixed_columns + columns_to_add]

all_columns = [x for x in X_reduce_te.columns.values if x not in ["Region_Code_0.0", "Region_Code_38.0", "Region_Code_52.0"]]



In [2]:
import optuna
np.seterr(all="ignore")

all_columns = [x for x in X_reduce_te.columns.values if x not in ["Region_Code_0.0", "Region_Code_38.0", "Region_Code_52.0"]]
transformer = ColumnTransformer(transformers=[('pwrtrans', PowerTransformer(), all_columns)])


def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1.0)
    max_depth = trial.suggest_int("max_depth", 2, 10)
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    subsample = trial.suggest_uniform('subsample', 0.4, 1.0)
    te_smoothing = trial.suggest_loguniform("te_smoothing", 0.05, 10.0)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 7)
    gamma = trial.suggest_uniform("gamma", 0.0, 1.0)
    colsample_bytree = trial.suggest_uniform("colsample_bytree", 0.3, 1.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-3, 1e2)

    
    xgb = XGBClassifier(random_state=42, eval_metric="logloss", learning_rate=learning_rate, max_depth=max_depth, n_estimators=n_estimators, subsample=subsample, min_child_weight=min_child_weight, gamma=gamma, colsample_bytree=colsample_bytree, reg_alpha=reg_alpha)
    encoder = TargetEncoder(cols=['Policy_Sales_Channel', 'Region_Code'], smoothing=te_smoothing)
    model = make_pipeline(encoder, transformer, xgb)

    score = cross_val_score(model, X_reduce_te, y, cv=10, scoring='roc_auc', n_jobs=-1).mean() # TODO: set cv=10 and the whole dataset
    print("  ROC AUC: {}".format(score))

    print("  Params for trial " + str(trial.number) + ": ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    
    return score

In [13]:
trials_number = 1 # TODO: set higer value

study_name = 'xgb-study'
storage_name = "sqlite:///{}.db".format(study_name)
study = optuna.create_study(study_name=study_name, direction="maximize", storage=storage_name)

study.enqueue_trial({
        "learning_rate": 0.05014352877833517,
        "max_depth": 7,
        "min_child_weight": 5,
        "n_estimators": 234,
        "subsample": 0.9009423215546716,
        "class_weight": None,
        "te_smoothing": 0.20145207364872467,
        "gamma": 0.2491618356357863,
        "colsample_bytree": 0.631803937308591,
        "reg_alpha": 0.025408324931767336
    })

optuna.logging.set_verbosity(optuna.logging.WARNING)
study.optimize(objective, n_trials=2)

print("\nBest trial:")
trial = study.best_trial

print("  ROC AUC: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

  ROC AUC: 0.8590493522316688
  Params for trial 0: 
    learning_rate: 0.05014352877833517
    max_depth: 7
    n_estimators: 234
    subsample: 0.9009423215546716
    te_smoothing: 0.20145207364872467
    min_child_weight: 5
    gamma: 0.2491618356357863
    colsample_bytree: 0.631803937308591
    reg_alpha: 0.025408324931767336
  ROC AUC: 0.8365101583894594
  Params for trial 1: 
    learning_rate: 0.0011408547793103705
    max_depth: 2
    n_estimators: 87
    subsample: 0.872274827571086
    te_smoothing: 2.7737921694052
    min_child_weight: 5
    gamma: 0.8941621038661537
    colsample_bytree: 0.5936983836843396
    reg_alpha: 2.7880713148493594

Best trial:
  ROC AUC: 0.8590493522316688
  Params: 
    colsample_bytree: 0.631803937308591
    gamma: 0.2491618356357863
    learning_rate: 0.05014352877833517
    max_depth: 7
    min_child_weight: 5
    n_estimators: 234
    reg_alpha: 0.025408324931767336
    subsample: 0.9009423215546716
    te_smoothing: 0.20145207364872467


In [15]:
import optuna
study_name = 'xgb-study'
storage_name = "sqlite:///{}.db".format(study_name)
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="maximize",)
study.optimize(objective, n_trials=100)

  ROC AUC: 0.8548510536519618
  Params for trial 2: 
    learning_rate: 1.171447845733699e-06
    max_depth: 7
    n_estimators: 151
    subsample: 0.5956965122416562
    te_smoothing: 0.12938532073884987
    min_child_weight: 1
    gamma: 0.0614557455025182
    colsample_bytree: 0.5678484025475484
    reg_alpha: 0.004983703091142559
  ROC AUC: 0.8483427679078958
  Params for trial 3: 
    learning_rate: 0.41064032199860423
    max_depth: 6
    n_estimators: 348
    subsample: 0.6487171329341139
    te_smoothing: 6.066659988561655
    min_child_weight: 2
    gamma: 0.6543065471891478
    colsample_bytree: 0.6299164530369348
    reg_alpha: 0.02148369026716384
  ROC AUC: 0.8491445513844462
  Params for trial 4: 
    learning_rate: 0.00025625592238240255
    max_depth: 10
    n_estimators: 321
    subsample: 0.5492090723913845
    te_smoothing: 5.867968850337677
    min_child_weight: 4
    gamma: 0.21501915035894026
    colsample_bytree: 0.36560040996361715
    reg_alpha: 0.10533087307543

/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sli

  ROC AUC: 0.8516931862087664
  Params for trial 9: 
    learning_rate: 9.294535824216191e-05
    max_depth: 8
    n_estimators: 207
    subsample: 0.7751932635294727
    te_smoothing: 0.6237764623674533
    min_child_weight: 6
    gamma: 0.7172766596492102
    colsample_bytree: 0.4108837527246837
    reg_alpha: 2.765207229513734


/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sli

/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/py

/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/py

/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home/jsowa/anaconda3/envs/healthnewsklearn/lib/py

  ROC AUC: 0.8559324238631698
  Params for trial 10: 
    learning_rate: 0.016290287896287295
    max_depth: 4
    n_estimators: 409
    subsample: 0.7762381901211387
    te_smoothing: 0.057371721726448596
    min_child_weight: 7
    gamma: 0.36504771836541167
    colsample_bytree: 0.4876975301312798
    reg_alpha: 67.05447303767998
  ROC AUC: 0.8589972062530837
  Params for trial 11: 
    learning_rate: 0.036542272962765805
    max_depth: 5
    n_estimators: 488
    subsample: 0.9872363299520472
    te_smoothing: 0.3522600407665465
    min_child_weight: 5
    gamma: 0.9592782847620096
    colsample_bytree: 0.7742444553853193
    reg_alpha: 1.0712508073655056
  ROC AUC: 0.8570262774240598
  Params for trial 12: 
    learning_rate: 0.018587552872626187
    max_depth: 4
    n_estimators: 486
    subsample: 0.9993580513744696
    te_smoothing: 0.48985712619598226
    min_child_weight: 5
    gamma: 0.5343990580432245
    colsample_bytree: 0.7775235309535705
    reg_alpha: 0.743176932407509

KeyboardInterrupt: 

In [3]:
import optuna
study_name = 'xgb-study'
storage_name = "sqlite:///{}.db".format(study_name)
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="maximize",)
study.optimize(objective, n_trials=100)

  ROC AUC: 0.8535110287214007
  Params for trial 15: 
    learning_rate: 0.003993462052981665
    max_depth: 5
    n_estimators: 173
    subsample: 0.8953349792182
    te_smoothing: 1.2269721773360744
    min_child_weight: 3
    gamma: 0.9990771853679652
    colsample_bytree: 0.6933005653662576
    reg_alpha: 0.29610458208646356
  ROC AUC: 0.858677144562259
  Params for trial 16: 
    learning_rate: 0.05866692581456225
    max_depth: 7
    n_estimators: 279
    subsample: 0.7739614962045015
    te_smoothing: 0.06929382239644655
    min_child_weight: 6
    gamma: 0.5148763179045638
    colsample_bytree: 0.985102117876606
    reg_alpha: 0.27869508027647494
  ROC AUC: 0.8549251015043341
  Params for trial 17: 
    learning_rate: 0.004850909035816609
    max_depth: 5
    n_estimators: 430
    subsample: 0.940482963838627
    te_smoothing: 0.23943443847743945
    min_child_weight: 3
    gamma: 0.002001439919436576
    colsample_bytree: 0.5155018578128264
    reg_alpha: 1.454163994834919


KeyboardInterrupt: 